Jin Jeon

HCDE 530 Computational Concepts

## Project 2b: Google Cloud API and Textual Data Analysis

### Introduction

#### Using Google Cloud Platform and its API : 
**Google Cloud Platform** is a suite of cloud computing services that lets developers interact with APIs that involve data storage, data analytics, and machine learning. In this notebook, I use GoogleSheets API to directly read in sheets from the drive. Besides reading in the sheets, the API allows you to create, manipulate, filter, and organize metadata. 

With GoogleSheets API, I study survey data collected from my previous projects in user research. The process involves establishing credentials and keys so the program knows to use my credentials to access the private files. 


#### Goal of this notebook: 

Survey studies are essential for understanding the users because they can be quickly developed and sent out to receive a good amount of sample in a short period of time. Surveys are powerful to be utilized for conducting initial research at a discovery stage to explore the general problem space and user behaviors. 

One of the free and efficient tools is the Google Forms. With GoogleSheets API, we are able to access and manipulate incoming or completed data through code. Furthermore, we can easily shape and analyze data through Python. 

While Google Form is capable of generating pie graphs and bar graphs to summarize the survey results, the visualizations are often rudimentary or basic. Using Python, we can easily handle large sets of data and breakdown and analyze the data by each demographic group, and probe for any interesting results.

**Specifically, let's see if there are any significant differences or responses that are made for each age group and gender.**

** _**Disclaimer:** Some of the code is an extension from my previous assignment._

## Running Sentiment Analysis using Google Natural Language API

### Code Implementation

In [1]:
# Imports the Google Cloud client library
import os
from google.cloud import language_v1

# set environment for credentials (need to be called with every start of instance)
# refer the reference tab for setting credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/Jin/google-cloud-sdk/natural-language-api.json"

# Instantiates a client
client = language_v1.LanguageServiceClient()

def analyze_sentiment(text):
    """Short summary.

    Parameters
    ----------
    text : str
        string of text to be analyzed

    Returns
    -------
    prints out sentiment score (-1, 1) and magnitude
    """
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)

    # Detects the sentiment of the text
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment

    print("Text: {}".format(text))
    print("Sentiment: {}, {}".format(sentiment.score, sentiment.magnitude))

### Quick glimpse at the sentiment analysis
Let's try feeding in some random sentences and see how sentiments turn out. 

In [2]:
# The text to analyze
text = u"The dish was delightfully delicious."
text2 = u"The overall experience was terrible."

analyze_sentiment(text)

analyze_sentiment(text2)

Text: The dish was delightfully delicious.
Sentiment: 0.8999999761581421, 0.8999999761581421
Text: The overall experience was terrible.
Sentiment: -0.800000011920929, 0.800000011920929


## Setting up Google Sheets API

The high level process of setting up Google API is as follows: 

**1. Create an Oauth credential** (Instruction link: https://developers.google.com/workspace/guides/create-credentials)


**2. Pip install the required packages** 

> ` pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib` 

**3. Set scope and spreadsheet ID**

 - **Auth scopes** set the permissions that your code requests to authorize for the app
For example, scope code of:
    - **.readonly** allows to read all resources and the meta data but with no writing operations
    - **.label** lets you create, read, update, and delete labels. 

- **Spreadsheet ID** is the parameter that gets used to tell which spreadsheet to access. The ID is called from a part of the full URL.

## Code Implementation

Please note that the following code will only run if you have your Google `credential.json` and `token.json` within the working directory. 

In [3]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = '11Den6g5nuR4B2CCUML1KrA0bEZXRpPZ7t83Ieyi7NJ4'

# Specify which sheet or row/column of data to call in
# refer to https://developers.google.com/sheets/api/guides/concepts#a1_notation for detail
RANGE_NAME = 'health_data'

creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API to read in the data
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId = SPREADSHEET_ID,
                            range = RANGE_NAME).execute()
values = result.get('values', [])

FileNotFoundError: [Errno 2] No such file or directory: 'token.json'

**We now have the sheet successfully loaded without having to open the Google Drive at all. We can use the data on the fly. Below, we will just confirm the data type that the data was read in.** 

In [ ]:
print(type(result))
print(type(values))

**Now let's convert it into a pandas dataframe so we can easily manipuate the data.**

In [ ]:
import pandas as pd

data = pd.DataFrame(values[1:], columns=values[0])

# let's confirm 
print(type(data))

data.shape

## Dataset
From the code above, we translated the data into pandas dataframe. Using `data.shape`, we know that there are total 27 questions collected from 71 participants. For simplicity, I remove any data that does not prefer to disclose gender. This brings the data size to 68. Due to the extensive length and branching logic within the survey, the data becomes more textual and qualitative for questions or columns in the back. I will primarily use selected columns that are of interest. 

Let's have a quick glance at the dataset. 

In [ ]:
# for simplicity, let's constrain the gender option to only two
gender_options = ['Man', 'Woman']
data = data[data['What is your gender?'].isin(gender_options)]

print('There are total ' + str(len(data)) + ' participants.')
print('The survey consists of ' + str(data.shape[1]) + ' questions (columns in the dataframe).')

# convert the column string values to integers
data['How would you rate your health?'] = data['How would you rate your health?'].astype(int)

data.head(3)

## Data Analysis
Now that we have seen the general dataframe structure, let's explore probing the data for analysis. 

In [ ]:
import os
import pandas as pd
from collections import Counter
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import warnings
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import scipy
from tabulate import tabulate

warnings.simplefilter(action='ignore', category=FutureWarning)  # suppress any warning
sns.set_color_codes('pastel')  # set color

### Querying data
Before we play around with data, let's query out the data that are of interest. This way we can manipulate the data more effectively without having to call on the entire dataset `data` everytime. 

There are total 7 different age groups. 

In [ ]:
# let's divde the data by gender first 
females = data.loc[data['What is your gender?'] == 'Woman']
males = data.loc[data['What is your gender?'] == 'Man']

# let's also create dataset divided by age group
age_under18 = data.loc[data['What age range are you?'] == 'Under 18']
age_18to24 = data.loc[data['What age range are you?'] == '18 - 24']
age_25to34 = data.loc[data['What age range are you?'] == '25 - 34']
age_35to44 = data.loc[data['What age range are you?'] == '35 - 44']
age_45to54 = data.loc[data['What age range are you?'] == '45 - 54']
age_55to64 = data.loc[data['What age range are you?'] == '55 - 64']
age_over65 = data.loc[data['What age range are you?'] == '65 or older']

In [ ]:
print(len(males))
print(len(females))

## Research Question: How does self-perception of health rating differ by gender and age?

Participants were asked, How would you rate your health? (5 being healthy, 1 being not healthy).

### A. Health rating by gender
Let's breakdown the data to see how self-perception of health wellness varies by gender and different age groups. In the code below, I first quary females and males from the data.

In [ ]:
# columns[4] is the column for health rating
mean_males = np.mean(males[males.columns[4]])
mean_females = np.mean(females[females.columns[4]])

print("Mean of males' self-health wellness: " + str(mean_males))
print("Mean of females' self-health wellness: " + str(mean_females))

### B. Health rating by age group

Now let's breakdown the data to see how self-perception of health wellness varies by different age groups.

In [ ]:
age = data.groupby('What age range are you?')['How would you rate your health?'].mean()
age

**Conversely, the age group 65 or older actually has the highest self-perception of wellness. The youngest group (age under 18) rated the lowest.**

### C. T-test for statistical signifcance
**However, with small samples of the two demographic groups `65 or older` and `Under 18`, we are not sure if the difference we see here is significant. Let's run a quick t-test to see if the difference we are seeing is statistically significant.** 

In [ ]:
t, p = scipy.stats.ttest_ind(age_over65['How would you rate your health?'], age_under18['How would you rate your health?'])

print('t: ' + str(t.round(4)))  
print('p: ' + str(p.round(4)))  # the p-val should be less than 0.05 in general to assume the difference we observe is signifcant

We see the p-value is 0.01 which is signifcant, which is one interesting find! So we can say that within this dataset, the people age over 65 perceive themselves to be more healthy than teenagers would do. 

### D. Health rating by age & gender group

Now let's breakdown by both gender and different age groups to look at how the self perception of health wellness change.

In [ ]:
age_gender = data.groupby(['What age range are you?', 'What is your gender?'])['How would you rate your health?'].mean().round(2)

age_gender

### E. Iteratively running t-test within each age group
We have several different age groups with each male and female gender group. Within each age group, let's run a t-test to see if there are any significant observed differences. 

In [ ]:
gender_options = ['Man', 'Woman']
age_groups = ['age_under18', 'age_18to24', 'age_25to34', 'age_35to44', 'age_45to54', 'age_55to64', 'age_over65']

table = []
table.append(['age group', 't value', 'p value'])

# iteratively run for t-tests within each age group defined in the list variable 'age_groups'
for i in range(0, len(age_groups)):
    data_string = "['How would you rate your health?']"
    eval_string1 = age_groups[i] + '.loc[' + age_groups[i] + "['What is your gender?'] == 'Man']"
    a = eval(eval_string1 + data_string)
    eval_string2 = age_groups[i] + '.loc[' + age_groups[i] + "['What is your gender?'] == 'Woman']"
    b = eval(eval_string2 + data_string)
#     strings_combined = 'scipy.stats.ttest_ind(a, b)'

    t, p = eval('scipy.stats.ttest_ind(a, b)')
    
    # we use a package called tabulate to print out a formatted table
    table.append([age_groups[i], t.round(4) ,p.round(4)])

print(tabulate(table, headers='firstrow'))

**We see that the p-values are all above 0.05 which means that there are no observed significant differences in gender within each age group.**

## Data Visualization

Let's first try plotting a simple visual violin plot. 

In [ ]:
age_plot = sns.catplot(x='What age range are you?', y='How would you rate your health?', \
                       hue='What is your gender?', kind="violin", data=data);

## Characterizing textual data through wordcloud
Let's change focus and try analyzing textual inputs from the participants. We will analyze the column How is your health and/or fitness information being used? question to identify any emerging keywords using the word cloud representation. Disclaimer: The result here is not such a useful or accurate representation as the stopwords did not clearly filter out.

We first call in a list of stopwords to filter out any unnecessary words, such as 'I', 'and', and etc. We then flatten out all the responses into a single list of words.

### Is there gender difference in how they use health data (text responses)? 

Participants were asked, "How is your health and/or fitness information being used?" Here, I try to breakdown the text data through representation of wordcloud, and see if there any characteristics found in each gender. 

In [ ]:
# builtin stopword sets from nltk 
stop = set(stopwords.words('english'))


def plot_wordcloud(df, col, separator=None): 
    """
    Plots a wordcloud of given dataframe and specific column. The text is counted at word level.
    
    Parameters
    ----------
    df: pandas dataframe
        dataframe that contains textual data
    col: int
        integer that points to the specific column with textual data 
    separator: str (default: None)
        string specified to breakdown the text by. Default is empty space
    
    Returns 
    -------
    Wordcloud plot 
    
    list of most common words in the dataframe
    
    """
    # filter out any NaNs
    response = [x for x in df[df.columns[col]] if x == x]
    # filter out any None
    response = [x for x in response if x != None]

    word_dict = []
    for i in range(0, len(response)):
        if separator == None:
            word_dict.append(response[i].split())
        else: 
            word_dict.append(response[i].split(separator))
    word_filtered = []

    # flatten the list and lower all letter cases
    for sublist in word_dict:
        for item in sublist:
            word_filtered.append(item.lower())

    # remove stopwords
    word_filtered = [x for x in word_filtered if x not in stop]

    word_filtered = [word.replace('.','').replace(',', '').replace("'",'') for word in word_filtered]
    
    # print most common words
    most_common_words = Counter(word_filtered).most_common(10)
    print(most_common_words)
    
    # plot wordcloud
    texts = " ".join(word for word in word_filtered)
    cloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(texts)
    plt.imshow(cloud, interpolation='bilinear')

In [ ]:
# plot wordcloud for Man
plot_wordcloud(males, 24)  # 24 specifies the column number

In [ ]:
# plot wordcloud for Woman
plot_wordcloud(females, 24)

The top image is the wordcloud of male participants and the bottom is that of female participants. We see that some words are not as meaningful and that one critical fault to this approach is that breaking down the responses into word level can misrepresent the meaning of their responses. For example, 'exercise' and 'not exercise' have two opposing ideas but here, it would count 'not' and 'exercise' as two seperate ideas. 

Even though the word counts are small, we see more 'food' and 'sleep' for female participants, leading to an assumption that it could be related to going on diets. 

### Analyzing categorical data using wordcloud

Participants were also asked, "what actions do you take regarding your health?" with multiple choices answer selections that include...
1. 'exercise'
2. 'take medication or health supplements'
3. 'track health or fitness'
4. 'learn more about health'
5. 'receive regular treatment at clinic'
6. 'maintain a diet'
7. 'receive mental counseling.' 

In [ ]:
plot_wordcloud(males, 3, ';')

In [ ]:
plot_wordcloud(females, 3, ';')

**From the two results above, we see that exercise is the most common practice for keeping up health in both genders. However, we see that in general, women tend to do more activities or attempts to maintain their health e.g. more frequently visits the clinic or receive counseling, whereas two men responded they simply do nothing at all.** 

## Conclusion & Thoughts
Wordcloud is a fun, engaging representation of textual data. However, more caution and consideration are needed because it can also tweak how the data is represented. For example, I coded the function so that it would breakdown any sentences or phrases into word level. This means that if someone does 'not exercise', it would still count 'exercise' and the end result would show 'exercise' being emphasized more. While the context of exercise is present, the meaning is totally the opposite.

### Future work
As much as I am interested in exploring textual data, I want to explore sentiment analysis and NLP models that could facilitate the analysis step of textual data. 

### References 

GoogleSheets API v4: https://developers.google.com/sheets/api/samples/reading

Google Oauth: https://developers.google.com/identity/protocols/oauth2/service-account#python

Google API Python Client: https://github.com/googleapis/google-api-python-client/blob/master/docs/oauth.md

Credentials: https://developers.google.com/workspace/guides/create-credentials
